In [ ]:
import pyautogui
from PIL import Image, ImageGrab
from collections import defaultdict, deque, Counter
import numpy as np
import cv2

In [ ]:
IMAGEPOS1 = (294, 453)
IMAGEPOS2 = (1240, 969)
COL = 11
ROW = 6
MAX_TURN = 2
BOXES = 35

In [ ]:
PIC_WIDTH = (IMAGEPOS2[0] - IMAGEPOS1[0]) / COL
PIC_HEIGHT = (IMAGEPOS2[1] - IMAGEPOS1[1]) / ROW
print(PIC_WIDTH, PIC_HEIGHT)

In [ ]:
def click(y, x):
    real_x = IMAGEPOS1[0] + x * PIC_WIDTH + PIC_WIDTH / 2
    real_y = IMAGEPOS1[1] + y * PIC_HEIGHT + PIC_HEIGHT / 2
    pyautogui.moveTo(real_x, real_y, duration=0.1)
    # pyautogui.sleep(0.2)
    pyautogui.click()

In [ ]:
def get_image():
    image = ImageGrab.grab(bbox=(IMAGEPOS1[0], IMAGEPOS1[1], IMAGEPOS2[0], IMAGEPOS2[1]))
    return image

In [ ]:
id_image = {}
for i in range(BOXES + 1):
    id_image[i] = Image.open(f"img/{i}.png")

In [ ]:
# cv template matching
def cv_match(img, template):
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    template = cv2.cvtColor(np.array(template), cv2.COLOR_RGB2BGR)
    res = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    match_area = img[max_loc[1]:max_loc[1]+template.shape[0], max_loc[0]:max_loc[0]+template.shape[1]]
    mean1 = cv2.mean(cv2.cvtColor(match_area, cv2.COLOR_BGR2HSV))[:3]
    mean2 = cv2.mean(cv2.cvtColor(template, cv2.COLOR_BGR2HSV))[:3]
    return max_val, -np.linalg.norm(np.array(mean1) - np.array(mean2))

In [ ]:
def get_gameboard(image):
    minigame_board = [[-1] * (COL + 2) for _ in range(ROW + 2)]
    cnter = defaultdict(list)
    split_image_dict = defaultdict(defaultdict)
    for i in range(ROW):
        for j in range(COL):
            small_image = image.crop((j * PIC_WIDTH, i * PIC_HEIGHT, (j + 1) * PIC_WIDTH, (i + 1) * PIC_HEIGHT))
            split_image_dict[i][j] = small_image
            score_list = []
            for k, v in id_image.items():
                cv_score = cv_match(small_image, v)
                if cv_score[0] > 0.7:
                    score_list.append((cv_score[0], cv_score[1], k))
            score_list.sort(key=lambda x: x[1], reverse=True)
            if score_list and score_list[0][0] > 0.7:
                cnter[score_list[0][2]].append((i + 1, j + 1))
                minigame_board[i+1][j+1] = score_list[0][2]
            if minigame_board[i+1][j+1] == -1:
                print("unknown image", i+1, j+1)
                # small_image.save(f"img/({i},{j}).png")
    return minigame_board, cnter, split_image_dict

In [ ]:
def move(board, st, ed):
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    def dfs(x, y, turn, dir):
        if x < 0 or x >= ROW + 2 or y < 0 or y >= COL + 2 or turn > MAX_TURN:
            return False
        if (x, y) == ed:
            board[st[0]][st[1]] = -1
            board[ed[0]][ed[1]] = -1
            click(st[0] - 1, st[1] - 1)
            click(ed[0] - 1, ed[1] - 1)
            return True
        if board[x][y] == BOXES or board[x][y] != -1:
            return False
        for next in directions:
            if next == dir:
                res = dfs(x + next[0], y + next[1], turn, next)
            else:
                res = dfs(x + next[0], y + next[1], turn + 1, next)
            if res:
                return True
        return False
    for dir in directions:
        if dfs(st[0] + dir[0], st[1] + dir[1], 0, dir):
            return True
    return False

In [ ]:
def copy_cnter(cnter):
    res = defaultdict(list)
    for k, v in cnter.items():
        res[k] = v.copy()
    return res

In [ ]:
def solve():
    image = get_image()
    gameboard, cnter, _ = get_gameboard(image)
    if BOXES in cnter:
        del cnter[BOXES]
    blocks = sum([len(v) for k, v in cnter.items()])
    cnt = 0
    while blocks:
        flag = True
        for i in range(BOXES):
            if len(cnter[i]) == 0:
                continue
            lst_cp = cnter[i].copy()
            flip = False
            for j in range(len(lst_cp)):
                for k in range(j + 1, len(lst_cp)):
                    if move(gameboard, lst_cp[j], lst_cp[k]):
                        print("click", lst_cp[j], lst_cp[k])
                        cnter[i].remove(lst_cp[j])
                        cnter[i].remove(lst_cp[k])
                        flag = False
                        flip = True
                        break
                if flip:
                    break
             # checki if the game is over
            try:    
                ok_pos = pyautogui.locateCenterOnScreen('./img/ok.png')
                if ok_pos:
                    pyautogui.click(ok_pos)
                    pyautogui.click(ok_pos)
                    return
                break
            except:
                pass
        if flag:
            # reget the gameboard for some blocks may be removed but not clicked
            image = get_image()
            gameboard, cnter, _ = get_gameboard(image)
            if BOXES in cnter:
                del cnter[BOXES]
            blocks = sum([len(v) for k, v in cnter.items()])
            cnt += 1
            # should not get too many times
            if cnt > 3:
                break

In [ ]:
def play():
    # use pyautogui to click the skip button
    skip_pos = pyautogui.locateCenterOnScreen('./img/skip.png')
    if skip_pos:
        pyautogui.click(skip_pos)
        pyautogui.click(skip_pos)
    # wait for the game to start
    pyautogui.sleep(13)
    solve()
    pyautogui.sleep(10)
    # use pyautogui to click the close button
    close_pos = pyautogui.locateCenterOnScreen('./img/close.png')
    if close_pos:
        pyautogui.click(close_pos)
        pyautogui.click(close_pos)
    pyautogui.sleep(5)

In [ ]:
while 1:
    play()